In [3]:
import Project_library as pl
import pickle as pkl

In [ ]:
# Load results
file_path = "../results_sweep"

with open(file_path, "rb") as file:
    results = pkl.load(file)
    # Print results
    for key, value in results.items():
        print(f"{key}: {value}")

EOFError: Ran out of input